<a href="https://colab.research.google.com/github/tejask666/Tejas_INFO5731_Spring2020/blob/master/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [0]:
import spacy
from nltk import word_tokenize
from itertools import islice
import numpy as np
import pandas as pd
from google.colab import files
from prettytable import PrettyTable
from textblob import TextBlob
from collections import Counter
#uploaded = files.upload()
import csv 
  
class Assignment_3():

  def get_data(self):
    cleaned_data=''
    with open('/content/testit.csv', 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            for row in csv_reader:
              cleaned_data=cleaned_data+str(row)
            return cleaned_data
  
  def calculate_ngram(self,cleaned_data,n):
          cleaned_data.replace("']['","").replace("['","").replace("']","")
          input_list= cleaned_data.split()
          mapped=zip(*[input_list[i:] for i in range(n)])
          counts = Counter(mapped)
          dictionary=dict(counts)
          dictionary={k: v for k, v in sorted(dictionary.items(), key=lambda item: item[1],reverse=True)}
          return list((dictionary.items()))
  
  def get_noun_phrases(self,sentence):
          nouns=''
          nlp = spacy.load("en_core_web_sm")
          doc = nlp(sentence)
          for token in doc:
             if token.pos_ == 'NOUN':
                 nouns=nouns+" "+str(token)
          return nouns

  def get_relative_probability(self,content):
          obj=Assignment_3()
          whole_data_list=content.split()
          sentence=content.split('][')
          print("relative probability of each noun =")
          pretty=PrettyTable()
          relative_prob_list=["Probability"]
          noun_list=[]
          noun=["Nouns"] 
          for i in range(len(sentence)):
            nouns_in_sentence=obj.get_noun_phrases(sentence[i]).split()
            if nouns_in_sentence != None:
                for j in range(len(nouns_in_sentence)):
                  noun_list.append(nouns_in_sentence[j])
                  p=noun_list.count(nouns_in_sentence[j])
                  if  p > 1:
                    pass
                  else:  
                    max_frequency=whole_data_list.count(nouns_in_sentence[j])
                    frequency=sentence[i].count(nouns_in_sentence[j])
                    if max_frequency != 0:
                      relative_prob=frequency/max_frequency
                      relative_prob_list.append(relative_prob)
                      noun.append(nouns_in_sentence[j])
          pretty=PrettyTable()
          pretty.field_names =noun
          pretty.add_row(relative_prob_list)
          print(pretty)
                      
  def calculate_word_count(self,word,csvdata):
          count=0
          csvdata.replace("']['","").replace("['","").replace("']","")
          content_list=csvdata.split()
          for i in range(len(content_list)):
            if content_list[i]==word:
               count=count+1
          return count
          
  def calculate_probability(self,two_grams_data,data):
          bigram_prob=[]
          obj=Assignment_3()
          dict_data=dict(two_grams_data)
          for i in (dict_data.items()):
            probability=i[1]/obj.calculate_word_count((i[0][0]),data)
            bigram_prob.append(str(i[0])+"/"+str(i[0][0])+" = "+ str(probability))
          print("probabilities for all the bigrams in the dataset= "+ str(bigram_prob)+"\n")
    

obj=Assignment_3()
content=obj.get_data()
# print("All 3-grams with frequencies:"+str(obj.calculate_ngram(content,3))+"\n")
# two_grams=obj.calculate_ngram(content,2)
# obj.calculate_probability(two_grams,content)
obj.get_relative_probability(content)

relative probability of each noun =
+-------------+--------+---------------------+---------------------+--------+--------+--------------------+------------+---------------------+--------------------+--------------------+---------------------+-----+--------------------+-----+----------+--------------------+---------------------+------+--------------------+--------+-------------+--------------------+--------------------+------+---------------------+---------------------+---------------------+---------------------+----------+-------+-------+--------------------+--------+-----------+---------------------+---------------------+----------------------+---------------------+-------+---------+---------------------+------+---------+----------------------+----------------------+---------------------+----------+-------------+--------------------+--------+-------------+------------+-------+--------------------+---------------------+---------------+----------+---------------------+------------------

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [0]:
import spacy
import nltk
from nltk import word_tokenize
from itertools import islice
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
from nltk.corpus import stopwords
from google.colab import files
from textblob import TextBlob
import numpy.linalg as LA
from collections import Counter
#uploaded = files.upload()
import csv 
  
class Assignment_3():

  def get_data(self):
    cleaned_data=''
    with open('/content/testit.csv', 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            for row in csv_reader:
              cleaned_data=cleaned_data+str(row)
            return cleaned_data
  def calculate_tf_idf(self,content):
    list_of_reviews = content.split("][")
    obj=Assignment_3()
    print("Printing documents-terms weights (tf*idf) matrix:"+"\n")
    print("Word          tf        idf       tf*idf")
    for i in range(len(list_of_reviews)): 
      list_of_reviews[i]=list_of_reviews[i].replace("['","").replace("']","").replace("'","")
      word=list_of_reviews[i].split()
      len_of_data=len(word)
      if i==20:
        break;
      for j in range(len(word)):
        term_in_document = word.count(word[j])
        normalized_tf = term_in_document / len_of_data 
        Idf=obj.calculate_idf(word[j],content)
        tf_idf=normalized_tf*Idf
        print('{:<10s}  {:>2f}  {:>2f}  {:>2f}'.format(str(word[j]),normalized_tf,Idf,tf_idf))

  def calculate_idf(self,word,content):
    count=0
    list_of_reviews=content.split('][')
    total_document=len(list_of_reviews)
    for i in range(len(list_of_reviews)):
       list_of_reviews[i]=list_of_reviews[i].replace("['","").replace("']","").replace("'","")
       if word in list_of_reviews[i]:
         count=count+1
    idf=np.log(total_document/count)
    return idf

  def get_cosine_similarity(self,cleaned_data,query):
    nltk.download('stopwords')
    print("Printing ranking of document using cosine similarity: "+"\n")
    data_set = cleaned_data.split("][")
    query= [query]
    stopWords = stopwords.words('english')
    vectorizer = CountVectorizer(stop_words = stopWords)
    transformer = TfidfTransformer()
    data_Vec_Array = vectorizer.fit_transform(data_set).toarray()
    query_Vec_Array = vectorizer.transform(query).toarray()
    cx = lambda a, b : np.inner(a, b)/(LA.norm(a)*LA.norm(b))
    cosine_sim = []
    for data_vec in data_Vec_Array:
        for query_vec in query_Vec_Array:
            cosine = cx(data_vec, query_vec)
            cosine_sim.append(cosine)
            #print(str(cosine))

    tf2=pd.DataFrame(data_set)
    new = tf2.filter(['Unnamed','clean_title'], axis=1)
    cosim = pd.Series(cosine_sim)
    review=pd.Series(data_set)
    new['Sentence']=review.values
    new['Cosine_similarity'] = cosim.values
    new.drop(new.loc[new['Cosine_similarity']==0].index, inplace=True)
    new["Rank"] = new["Cosine_similarity"].rank().astype(int)
    new.sort_values("Cosine_similarity", inplace = True) 
    print(new)
    

obj=Assignment_3()
content=obj.get_data()
query = "As a viewer that actually went to TIFF and witnessed this film and didn't want to believe the hype, it is an absolute MASTERPIECE and Phoenix is a certified legend."
obj.get_cosine_similarity(content,query)
obj.calculate_tf_idf(content) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Printing ranking of document using cosine similarity: 

                                             Sentence  Cosine_similarity  Rank
88  "3/10\nWent into this blind, and i didnt enjoy...           0.012669     1
72  "10/10\nMasterpiece\njegorstepanow4 October 20...           0.015082     2
50  "10/10\nTHIS IS HOW A DC MOVIE SHOULD BE!\nlil...           0.017547     3
31  '10/10\nThe Rise of the Joker!\nahmetkozan13 N...           0.020870     4
45  '10/10\nAstonishing Masterpiece\nSinbaddylad4 ...           0.020870     4
..                                                ...                ...   ...
30  '10/10\nRequiem of a broken man\nandymaranam1 ...           0.162088    70
27  "7/10\nIs it great? I don't know.\nernestoherr...           0.166957    71
48  "3/10\nI made an account just to rate my disap...           0.183942    72
23  "7/10\nBetween a 7 and an 8\nro

# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file

 Link: httpsgithub.comtejask666Tejas_INFO5731_Spring2020blobmasterTraining%20and%20Evaluation%20Data%20.csv